# irodsConnector 

## Create an iRODS session (connection to iRODS server)

In [2]:
from ibridges import Session
import os, json
from getpass import getpass

### Password authentication

In [ ]:
with open(os.path.expanduser("~/.irods/irods_environment.json"), "r") as f:
    ienv = json.load(f)
password = getpass("Your iRODS password")
session = Session(irods_env=ienv, password=password)

### Using the cached password ~/.irods/.rodsA

In [ ]:
session = Session(irods_env_path=os.path.expanduser("~/.irods/irods_environment.json"))

### Checking some session parameters

In [ ]:
print(session.username)
print(session.default_resc) # the resource to which data will be uploaded
print(session.zone) # default home for iRODS /zone/home/username
print(session.server_version)

## Metadata of data objects and collections

### Retrieve an iRODS object or collection and list its metadata

In [ ]:
#print(ienv)
coll_path = ienv.get('irods_home', '') + '/books/BenHur.txt'
print(coll_path)
coll = session.irods_session.data_objects.get(coll_path) # TODO: exchange once data_ops is done

In [ ]:
from ibridges.irodsconnector.meta import MetaData
coll_meta = MetaData(coll)
print(coll_meta)

### View, add, set and delete metadata

In [ ]:
coll_meta.add('NewKey', 'NewValue')
coll_meta.add('NewKey', 'AnotherValue')
print(coll_meta)

Note, that keys are always capitalised. This is good practice in iRODS.

In [ ]:
coll_meta.delete('NEWKEY', 'NewValue')
print(coll_meta)

We can also set the meta data to a single key, value, units pair. This will remove any other entries with the same key.

In [ ]:
coll_meta.set("NEWKEY", "YetAnotherValue")
print(coll_meta)

### Accessing metadata 
With the orint function you can quickly inspect the metadata of an iRODS collection or object. If you want to extract and do something with the metadata, use the `__iter__` function. We give a small example below where we assume that the metadata contains a key/name *AUTHOR*:

In [ ]:
for md in coll_meta.__iter__():
    if md.name == 'AUTHOR':
        print(coll_meta.item, "was written by", md.value)

## Resources and handling resources

In [ ]:
from ibridges.irodsconnector.resources import Resources
resources = Resources(session)

### Check if default resource exists

In [ ]:
default_resc = resources.get_resource(session._irods_env.get("irods_default_resource", ""))
print(default_resc.name)
print(default_resc.free_space) # Metadata how much bytes are left on resource, None if not set
print(default_resc.type) # Storage policy

### Listing resources

In [ ]:
print(resources.resources()) # all resources
print(resources.root_resources) # all writeable resources (name, status, free space, context)

### Retrieve current free space
In contrast to `resc.free_space` the function `get_free_space` accumulates all free space in the subtree starting with the resource as parent.

In [ ]:
resources.get_free_space(session._irods_env.get("irods_default_resource", "")) # default resource name

## Tickets (access string to collection or data object)

### List all tickets which you issued

In [ ]:
from ibridges.irodsconnector.tickets import Tickets
tickets = Tickets(session)
print(tickets.all_ticket_strings)
print(tickets.all_tickets()) # (ticket string, access mode, object or collection id, expiry date in epoche)

### Issue a ticket

In [ ]:
from datetime import datetime

exp_date = datetime.today().strftime('%Y-%m-%d.%H:%M:%S')
objPath = "/nluu12p/home/research-test-christine/books/BenHur.txt"
ticket = tickets.create_ticket(obj_path=objPath, ticket_type="write", expiry_string=exp_date) # allow write access

In [ ]:
tickets.all_ticket_strings

### Fetch and delete a ticket

In [ ]:
ticket = tickets.get_ticket(tickets.all_ticket_strings[0])
tickets.delete_ticket(ticket)

## Rules
Execute an iRODS rule from a rule file:

In [ ]:
from ibridges.irodsconnector.rules import Rules
rules = Rules(session)
rule_file = "example_rules/example.r"
stdout, stderr = rules.execute_rule(rule_file, {})

In [ ]:
print(stdout)
print(stderr)

### Overwrite parameters in iRODS rules
iRODS rule files end with a line like `input *in="This is a string or a path or etc"`. In this example there is an input parameter called `'*in'` and it takes the value `"This is a string or a path or etc"`. We can overwrite these values by passing a python dictionary:

In [ ]:
params = {'*in': '"Another input"'}
stdout, stderr = rules.execute_rule(rule_file, params)
print(stdout)

Changing the type of the parameter from str to int, you can also see that keys in the dictionary which do not correspond to an input parameter, are simply ignored.

In [ ]:
params = {'*in': 4, '*another_val': '"Value"'}
stdout, stderr = rules.execute_rule(rule_file, params)
print(stdout)

## Permissions
### Reading and setting access for objects (files)

In [23]:
from ibridges.irodsconnector.permission import Permission

def show_permissions(obj, user=None):
    for perm in obj:
        if user is None or user and user==perm.user_name:
            print(f"{perm.path} {perm.access_name:<13} {perm.user_name} ({perm.user_type}) {perm.user_zone}")

# define a user to set permissions for
my_user='my_user'

# select a file to set permissions on
obj_path = ienv.get('irods_home', '') + '/books/Dracula.txt'

print(obj_path)

obj = session.irods_session.data_objects.get(obj_path) # TODO: exchange once data_ops is done

/nluu12p/home/research-test-christine/books/Dracula.txt


In [24]:
# create Permission object for the file
obj_perm = Permission(session, obj)

# see the current permissions set on the file
show_permissions(obj_perm)

# remember current access level to the file for my_user
original_permission=None
perms=[x for x in obj_perm if x.user_name==my_user]
if len(perms)>0:
    original_permission=perms[0].access_name

/nluu12p/home/research-test-christine/books/Dracula.txt read object   datamanager-its (rodsgroup) nluu12p
/nluu12p/home/research-test-christine/books/Dracula.txt read object   m.d.schermer@uu.nl (rodsuser) nluu12p
/nluu12p/home/research-test-christine/books/Dracula.txt read object   read-test-christine (rodsgroup) nluu12p
/nluu12p/home/research-test-christine/books/Dracula.txt own           research-test-christine (rodsgroup) nluu12p
/nluu12p/home/research-test-christine/books/Dracula.txt own           rods (rodsadmin) nluu12p


In [25]:
# define a new access level
print("current: ", original_permission)
new_permission='modify_object' if original_permission in ['read object', 'read_object'] else 'read_object'
print("new: ", new_permission)

# set the new access level
obj_perm.set(new_permission, my_user)

# see the updated access level for my_user
show_permissions(obj_perm)

current:  read object
new:  modify_object
/nluu12p/home/research-test-christine/books/Dracula.txt read object   datamanager-its (rodsgroup) nluu12p
/nluu12p/home/research-test-christine/books/Dracula.txt modify object m.d.schermer@uu.nl (rodsuser) nluu12p
/nluu12p/home/research-test-christine/books/Dracula.txt read object   read-test-christine (rodsgroup) nluu12p
/nluu12p/home/research-test-christine/books/Dracula.txt own           research-test-christine (rodsgroup) nluu12p
/nluu12p/home/research-test-christine/books/Dracula.txt own           rods (rodsadmin) nluu12p


In [26]:
# available access levels:
print(obj_perm.available_permissions.keys())

# several synonyms can be used:
obj_perm.set('read', my_user)
obj_perm.set('read object', my_user)
obj_perm.set('read_object', my_user)

obj_perm.set('write', my_user)
obj_perm.set('modify object', my_user)
obj_perm.set('modify_object', my_user)
obj_perm.set('modify_object', my_user)

obj_perm.set(original_permission, my_user)

['null', 'read object', 'modify object', 'own']


### Reading and setting access for collections (folders)

In [27]:
# select a file to set permissions on
coll_path = ienv.get('irods_home', '') + '/books/'

print(coll_path)

coll = session.irods_session.collections.get(coll_path) # TODO: exchange once data_ops is done
coll_perm = Permission(session, coll)

/nluu12p/home/research-test-christine/books/


In [30]:
# see all access levels for the collection for your user
show_permissions(coll_perm, my_user)

/nluu12p/home/research-test-christine/books own           m.d.schermer@uu.nl (rodsuser) nluu12p
/nluu12p/home/research-test-christine/books modify object m.d.schermer@uu.nl (rodsuser) nluu12p


In [31]:
# set new access level
coll_perm.set('read', my_user)
show_permissions(coll_perm, my_user)

/nluu12p/home/research-test-christine/books own           m.d.schermer@uu.nl (rodsuser) nluu12p
/nluu12p/home/research-test-christine/books read object   m.d.schermer@uu.nl (rodsuser) nluu12p


In [44]:
# in addition, collections support 'inherit' and 'noinherit' permissions (link to documentation?)
# current inheritance state
coll.inheritance

True

In [45]:
# set new value
coll_perm.set('noinherit' if coll.inheritance else 'inherit')

# old value is cached in the session, retrieve anew to see change
coll = session.irods_session.collections.get(coll_path) # TODO: exchange once data_ops is done
coll.inheritance

False